In [ ]:
#imports

!pip install keras-tuner
!pip install scikeras
import json
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow import keras
import keras_tuner
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from keras.utils import to_categorical

In [ ]:
#read_csv

data = pd.read_csv('/content/drive/MyDrive/DataFrame.csv', index_col = 'Unnamed: 0' )

In [ ]:
#data_for_model

N = 2000 #number_of_rows
seq_len = 10 #size_of_window
number_of_unique_chords = 193
row_len = 100
batch_len = 7


X_dur = np.zeros((N*seq_len, 4))
X_chord = np.zeros((N*seq_len, 5))
Y = np.zeros((N*seq_len, 1))


for i in range(N*seq_len):

  row = data.iloc[i//seq_len]

  for k in range(0, row_len, seq_len):
    x = row[k:k+9] #slice
    y = row[k+9:k+10]

    for j in range(0, len(x)-1, 2):
        chord = x[j]
        duration = x[j+1]
        X_chord[i][j//2] = chord
        X_dur[i][j//2] = duration
    X_chord[i][4] = x[8]
    Y[i] = y[0]

X_dur = (X_dur - X_dur.mean()) / X_dur.std()
X = np.empty((N*seq_len, 9, 1))

for i in range(N*seq_len):
  for j in range(0, 8, 2):
      a = X_chord[i][j//2]
      b = X_dur[i][j//2]
      X[i][j] = a
      X[i][j+1] = b
  X[i][8] = X_chord[i][4]


X_train, X_validation, y_train, y_validation = train_test_split(X, Y, random_state=42, test_size=0.2, shuffle=True)

In [ ]:
#model

model = Sequential()
model.add(LSTM(64, activation = 'relu', input_shape = (9, 1)))
model.add(Dropout(0.4))

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(1, activation = 'relu'))

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate,
    decay_steps=100,
    decay_rate=0.9)

opt = tf.keras.optimizers.Adam(
    learning_rate = lr_schedule
)

model.compile(optimizer = opt, loss = 'mse', metrics = ['mse'])
history = model.fit(X_train, y_train, validation_split = 0.2, epochs = 40, batch_size = 16)

In [ ]:
#test_for_one_chord

t = np.reshape(X_validation[333], (1, 9, 1)) #you_can_choose_index_for_X_validation
predicted_chord = model.predict(t)
print(predicted_chord)

1/1 [==============================] - 0s 130ms/step
[[2.2127]]


In [ ]:
#decode, encode, convert and filler
from random import shuffle
def decode_chord_data(chord_data):
    file = open("/content/drive/MyDrive/Decode.json")
    decoder = json.load(file)
    file.close()

    res = []
    for chord, duration in chord_data:
        res.append((decoder[str(chord)], duration))

    return res


def encode_chord_data(chord_data):
    file = open("/content/drive/MyDrive/Decode.json")
    decoder = json.load(file)
    file.close()

    encoder = dict()
    for key, value in decoder.items():
        encoder[value] = int(key)

    res = []
    for chord, duration in chord_data:
        res.append((encoder[chord], duration))

    return res


def convert(arr):
  print(len(arr))
  new_arr = np.zeros((1, 9, 1))
  for i in range(0, len(arr)//2-1, 2):
    new_arr[0][i][0] = arr[i]
    new_arr[0][i][0] = arr[i+1]
  new_arr[0][8][0] = arr[8]
  return new_arr

def filler(arr):
    for_model = []
    while(len(for_model) < 8):
        for_model += arr
    for_model = for_model[len(for_model)-9:len(for_model)-1]
    return for_model

def shuffle_for_pair(arr):
  new_arr = []
  for i in range(0, len(arr)-1, 2):
    new_arr.append((arr[i], arr[i+1]))
  shuffle(new_arr)
  y = []
  for i in new_arr:
    y.append(i[0])
    y.append(i[1])
  return y

def filler_for_chord(arr):
  for_model = []
  while(len(for_model) < 98):
      for_model += arr
  return for_model[:98]

In [ ]:
#predict

def predict_sequence(chord_data, n):
  chord_data = encode_chord_data(chord_data)
  sequence_for_chord = [i for p in chord_data for i in p]
  sequence_for_duration = [i for p in chord_data for i in p]
  result = list(chord_data)

#duration
  seq_dur = []
  if len(sequence_for_duration) > 8:
        sequence_for_duration = sequence_for_duration[len(sequence_for_duration)-9:len(sequence_for_duration)-1]
  elif len(sequence_for_duration) < 8:
      sequence_for_duration = filler(sequence_for_duration)
  for i in range(0, len(sequence_for_duration)-1, 2):
        chord = sequence_for_duration[i]
        duration = sequence_for_duration[i+1]
        seq_dur.append(chord)
        seq_dur.append(duration)

#chord
  if len(sequence_for_chord) > 98:
      sequence_for_chord = sequence_for_chord[-98:]
  elif len(sequence_for_chord) < 98:
      sequence_for_chord = filler_for_chord(sequence_for_chord)

  for i in range(n):
    predicted_chord = int(model_for_chord.predict(data = sequence_for_chord))
    seq_dur.append(predicted_chord)
    seq_for_predict = convert(seq_dur)
    predicted_duration = model.predict(seq_for_predict)[0][0]
    seq_dur.append(predicted_duration)
    sequence_for_chord.append(predicted_chord)
    sequence_for_chord.append(predicted_duration)
    sequence_for_chord = sequence_for_chord[2:]
    seq_dur = seq_dur[2:]
    result.append((predicted_chord, predicted_duration))


  return decode_chord_data(result)

a = [("Am", 2.0), ("F", 3.4), ("Dm", 4.5), ("E", 1.2), ("Fm", 1.55), ("Cm", 2.44)]